In [19]:
import sys
sys.path.append('../..')  # Adjust the path to include the parent directory

import numpy as np
from qiskit import QuantumCircuit, qasm2
from qiskit.circuit.library import QFT
from qiskit.transpiler import PassManager
import canopus
import uuid
import os
import pytket.qasm

In [20]:
def canopus_pass(qc, topology, isa):
    if topology == "chain":
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    backend = canopus.CanopusBackend(coupling_map, isa)
    qc_mapped = PassManager(canopus.CanopusMapping(backend)).run(qc)
    return qc_mapped

def sabre_pass(qc, topology):
    if topology == "chain":
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    backend = canopus.CanopusBackend(coupling_map)
    qc_mapped = PassManager(canopus.SabreMapping(backend)).run(qc)
    return qc_mapped


def toqm_pass(qc, topology):
    TOQM_FLAGS = '-defaults -latency Latency_1_2_6 -expander GreedyTopK 10 -queue TrimSlowNodes 2000 1000 -nodeMod GreedyMapper -retain 1'

    input_fname = f"{uuid.uuid4().hex}.qasm"
    qasm2.dump(qc, input_fname)

    if topology == "chain":
        coupling_file = '../../configs/chain.txt'
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_file = '../../configs/hhex.txt'
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_file = '../../configs/square.txt'
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    # create a tmp file
    output_fname = f"{uuid.uuid4().hex}.qasm"

    coupling_fname = f'./tmp_{uuid.uuid4().hex}.txt'
    graph = coupling_map.graph.to_undirected()
    with open(coupling_fname, 'w') as f:
        f.write('{} {}\n'.format(graph.num_nodes(), graph.num_edges()))
        for src, dst in graph.edge_list():
            f.write('{} {}\n'.format(src, dst))

    os.system(f'../mapper {input_fname} {coupling_fname} {TOQM_FLAGS} > {output_fname}')
    os.remove(coupling_fname)  # Clean up temporary coupling file

    # Replace 'swp ' with 'swap ' in the output file
    with open(output_fname, 'r') as f:
        content = f.read()
    content = content.replace('swp ', 'swap ')
    with open(output_fname, 'w') as f:
        f.write(content)

    qc_toqm = QuantumCircuit.from_qasm_str(content)

    # remove the temporary files
    os.remove(input_fname)
    os.remove(output_fname)
    return qc_toqm



In [21]:
cx_cost_est = canopus.SynthCostEstimator('cx')
sqisw_cost_est = canopus.SynthCostEstimator('sqisw')

┌──────────────────┐                                                                ┌──────────────────┐                  ┌─────────────────┐   ┌───────────────────┐   ┌────────────────┐               
q_0: ─┤ U(7π/2,0,11.192) ├────────────────────────────────────────────────────────────────┤0                 ├──────────────────┤0                ├───┤ U(7π/8,-π/2,3π/2) ├───┤0               ├───────────────
      └┬────────────────┬┘                  ┌─────────────────┐                           │                  │┌────────────────┐│                 │┌──┴───────────────────┴──┐│  Can(0.25,0,0) │┌─────────────┐
q_1: ──┤ U(7π/2,0,5π/8) ├───────────────────┤0                ├───────────────────────────┤                  ├┤0               ├┤  Can(0.125,0,0) ├┤ U(11π/3,0.61548,6.8987) ├┤1               ├┤ U(3π/2,0,π) ├
       ├───────────────┬┘ ┌────────────────┐│                 │┌─────────────────────────┐│  Can(0.0625,0,0) ││  Can(0.25,0,0) ││                 │└──────┬────────────┬─────┘└────────────────┘└─────────────┘
q_2: ──┤ U(3π/2,0,π/2) ├──┤0               ├┤  Can(0.125,0,0) ├┤ U(10.284,9.0593,4.1825) ├┤                  ├┤1               ├┤1                ├───────┤ U(π/2,0,π) ├───────────────────────────────────────
     ┌─┴───────────────┴─┐│  Can(0.25,0,0) ││                 │└─────────────────────────┘│                  │└┬──────────────┬┘└─────────────────┘       └────────────┘                                       
q_3: ┤ U(4.516,-π/2,π/2) ├┤1               ├┤1                ├───────────────────────────┤1                 ├─┤ U(3π/2,0,4π) ├────────────────────────────────────────────────────────────────────────────────
     └───────────────────┘└────────────────┘└─────────────────┘                           └──────────────────┘ └──────────────┘

In [22]:
n = 5
qc_ori = QFT(n, do_swaps=False).decompose()
qc_cx = canopus.logical_optimize(qc_ori)
qc = canopus.rebase_to_tk2(qc_ori)
qc.draw(fold=10000)

┌──────────────────┐                                                ┌──────────────────┐                                                                                                          
q_0 -> 0 ─┤ U(7π/2,0,11.192) ├────────────────────────────────────────────────┤0                 ├────────────────────────────X─────────────────────────────────────────────────────────────────────────────
          └┬────────────────┬┘                      ┌─────────────────┐       │  Can(0.0625,0,0) │      ┌──────────────┐      │ ┌─────────────────┐┌───────────────────┐                                    
q_1 -> 1 ──┤ U(7π/2,0,5π/8) ├───────────────────────┤0                ├─────X─┤1                 ├──────┤ U(3π/2,0,4π) ├──────X─┤0                ├┤ U(7π/8,-π/2,3π/2) ├─X──────────────────────────────────
         ┌─┴────────────────┴┐┌────────────────┐    │  Can(0.125,0,0) │     │ └┬────────────────┬┘┌─────┴──────────────┴────┐   │  Can(0.125,0,0) │└───┬────────────┬──┘ │ ┌────────────────┐               
q_3 -> 2 ┤ U(4.516,-π/2,π/2) ├┤0               ├────┤1                ├─────X──┤0               ├─┤ U(11π/3,0.61548,6.8987) ├─X─┤1                ├────┤ U(π/2,0,π) ├────X─┤0               ├───────────────
         └─┬───────────────┬─┘│  Can(0.25,0,0) │┌───┴─────────────────┴───┐    │  Can(0.25,0,0) │ └─────────────────────────┘ │ └─────────────────┘    └────────────┘      │  Can(0.25,0,0) │┌─────────────┐
q_2 -> 3 ──┤ U(3π/2,0,π/2) ├──┤1               ├┤ U(10.284,9.0593,4.1825) ├────┤1               ├─────────────────────────────X────────────────────────────────────────────┤1               ├┤ U(3π/2,0,π) ├
           └───────────────┘  └────────────────┘└─────────────────────────┘    └────────────────┘                                                                          └────────────────┘└─────────────┘

### Chain tolology

In [25]:
qc_canopus = canopus_pass(qc, "chain", "sqisw")
qc_canopus.draw(fold=10000)

sh: line 1: 29546 Segmentation fault: 11  ../mapper 7c9af22e1eb344c19e5fcaf5aa790bf9.qasm ../../configs/chain.txt -defaults -latency Latency_1_2_6 -expander GreedyTopK 10 -queue TrimSlowNodes 2000 1000 -nodeMod GreedyMapper -retain 1 > 6d76bbda9546475bb5b5285c87b917ae.qasm


In [ ]:
qc_sabre = sabre_pass(qc, "chain")
qc_sabre.draw(fold=10000)

In [ ]:
qc_toqm = toqm_pass(qc_cx, "chain")
qc_toqm.draw(fold=10000)

In [ ]:
cx_cost_est.eval_circuit_cost(qc_sabre), cx_cost_est.eval_circuit_cost(qc_canopus), cx_cost_est.eval_circuit_cost(canopus.rebase_to_canonical(qc_toqm))

In [ ]:
cx_cost_est.eval_circuit_cost(qc_sabre), cx_cost_est.eval_circuit_cost(qc_canopus), cx_cost_est.eval_circuit_cost(canopus.rebase_to_canonical(qc_toqm))


In [ ]:
cx_cost_est.eval_circuit_cost(canopus.rebase_to_canonical(qc_canopus))

In [ ]:
(qc_canopus).draw('mpl', style='clifford', fold=1000)

In [ ]:
(qc_canopus).draw(fold=10000)

In [ ]:
(qc_toqm.reverse_bits()).draw('mpl', fold=1000)

In [ ]:
qc_cx.draw(fold=1000)

In [ ]:
qc_canopus = canopus_pass(qc, "square", "cx")
qc_canopus.draw(fold=10000)

In [ ]:
cx_cost_est.eval_circuit_cost(qc_canopus)

In [ ]:
qc_toqm = toqm_pass(qc_cx, "square")

In [ ]:
cx_cost_est.eval_circuit_cost(canopus.rebase_to_canonical(qc_toqm))

In [ ]:
qc_toqm.draw(fold=10000)

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qc_toqm)).depth()

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qc_canopus)).depth()


In [ ]:
qft_n18 = pytket.qasm.circuit_from_qasm('../output/logical/tk2/qft_n18.qasm')
qft_n18 = canopus.utils.tket_to_qiskit(qft_n18)
qft_n18.draw(fold=10000)

qft_n18_toqm = toqm_pass(QuantumCircuit.from_qasm_file('../output/logical/cx/qft_n18.qasm'), "chain")
qft_n18_canopus = canopus_pass(qft_n18, "chain", "cx")

qft_n18_toqm = canopus.rebase_to_canonical(qft_n18_toqm)
qft_n18_canopus = canopus.rebase_to_canonical(qft_n18_canopus)

In [ ]:
qft = QFT(18, do_swaps=False).decompose()

qft_toqm = toqm_pass(canopus.logical_optimize(qft), "chain")
qft_canopus = canopus_pass(canopus.rebase_to_tk2(qft), "chain", "cx")

qft_toqm = canopus.rebase_to_canonical(qft_toqm)
qft_canopus = canopus.rebase_to_canonical(qft_canopus)

In [ ]:
canopus.utils.print_circ_info(qft_n18_toqm)
canopus.utils.print_circ_info(qft_n18_canopus)

In [ ]:
cx_cost_est.eval_circuit_cost((qft_canopus))


In [ ]:
cx_cost_est.eval_circuit_cost(canopus.rebase_to_canonical(qft_toqm))


In [ ]:
qft_n18_canopus

In [ ]:
cx_cost_est.eval_circuit_cost((qft_canopus))

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_n18_toqm)).draw(fold=10000)

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_toqm)).depth()

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_n18_toqm)).depth()

In [ ]:
cx_cost_est.eval_circuit_cost(canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_n18_toqm)))

In [ ]:
cx_cost_est.eval_circuit_cost(canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_toqm)))

In [ ]:
from accel_utils import *

In [ ]:
synth_cost_by_cx(0.5, 0, 0)

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_toqm)).depth()

In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_n18_toqm)).draw(fold=1000)


In [ ]:
canopus.utils.remove_1q_gates(canopus.rebase_to_canonical(qft_toqm)).draw(fold=1000)

In [ ]:
def nnn(n):
    return n * (n - 1) // 2

nnn(16)

In [ ]:
CouplingMap.from_grid(3, 3).draw()

In [ ]:
canopus.utils.gene_square_coupling_map(12).draw()